In [1]:
import folium
import os

In [2]:
#Data import
data = os.path.join('Subway_Stations.geojson')

In [3]:
#Create a map object, focused on NYC:
m = folium.Map(location=[40.7142700, -74.0059700], zoom_start=13)

In [4]:
folium.GeoJson(data, name='Train Stations-Vor').add_to(m)

In [5]:
import pandas as pd
from scipy.spatial import Voronoi, voronoi_plot_2d

In [6]:
#File with station locations in NYC:
libs = pd.read_csv('DOITT_SUBWAY_STATION_01_13SEPT2010.csv')

#Create lists to hold coordinates and popups:
coords = []
popups = []
icons = []

In [7]:
#For each row in the CSV, pull out the latitude, longitude, and name of the library:
for index, row in libs.iterrows():
    words = row['the_geom'].split(" ")
    lat = float(words[2][:-1])
    lon = float(words[1][1:])
    name = row['NAME']
    #Add the [lat,lon] to list of coordinates:
    coords.append([lat,lon])
#Use scipy to make the voronoi diagram:
vor = Voronoi(coords, qhull_options='Qbb Qc Qx')

In [8]:
#Use geojson file to write out the features
from geojson import FeatureCollection, Feature, Polygon

In [9]:
#The output file, to contain the Voronoi diagram we computed:
vorJSON = open('Subway_Stations.geojson', 'w')
point_voronoi_list = []
feature_list = []

In [10]:
for region in range(len(vor.regions)-1):
    vertex_list = []
    for x in vor.regions[region]:
        if x == -1:
            break;
        else:
            #Get the vertex out of the list, and flip the order for folium:
            vertex = vor.vertices[x]
            vertex = (vertex[1], vertex[0])
        vertex_list.append(vertex)
    #Save the vertex list as a polygon and then add to the feature_list:
    polygon = Polygon([vertex_list])
    feature = Feature(geometry=polygon, properties={})
    feature_list.append(feature)

In [11]:
#Write the features to the new file:
feature_collection = FeatureCollection(feature_list)
print (feature_collection, file=vorJSON)
vorJSON.close()

In [12]:
folium.GeoJson(data, name='Train Stations-Vor').add_to(m)

In [13]:
m

In [14]:
m.save('Map.html')